In [ ]:
#Named entity recognition app
# !pip install transformers
# !pip install gradio
from transformers import pipeline
import gradio as gr

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Andrew and I live in California", "My name is Poli and work at HuggingFace"])
demo.launch()


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Closing server running on port: 7860
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
from torch.optim.adamw import AdamW
!pip install transformers
!pip install datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
checkpoint="bert-base-uncased"
Tokenizer=AutoTokenizer.from_pretrained(checkpoint)
Model= AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequences= [ "I've been waiting for a HuggingFace course my whole life.", "This course is amazing!"]

batch=Tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

print(batch)
batch["labels"] = torch.tensor([1, 1])
# print(batch)
optimizer= AdamW(Model.parameters())
loss=Model(**batch).loss  #forward flow
loss.backward() # backward flow
optimizer.step()







In [ ]:
!pip install datasets
from datasets import load_dataset
new_dataset=load_dataset("glue","mrpc")
print(new_dataset)
# new_dataset["train"][5]
train_dataset=new_dataset["train"]
train_dataset.features

# print(type(new_dataset),type(train_dataset))


In [ ]:
# #dataset preprocessing

# checkpoint="bert-base-uncased"
# Tokenized=Tokenizer("This app does not look good","Hugging face is the app", padding=True, truncation =True)
# print(Tokenized)

# Tokenizer.convert_ids_to_tokens(Tokenized['input_ids'])

#15th of training set

train_dataset[15]

Tokenize_15_separate=Tokenizer(train_dataset[15]['sentence1'], padding=True, truncation=True)

Tokenize_15_together=Tokenizer(train_dataset[15]['sentence1'],train_dataset[15]['sentence2'], padding=True, truncation=True)
print(Tokenize_15_separate)
print(Tokenize_15_together)



{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


###Preprocessing and fine tuning

In [ ]:
#preprocessing
!pip install accelerate -U
!pip install transformers[torch]
!pip install datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset

data=load_dataset('glue','mrpc')
checkpoint="bert-base-uncased"
Tokenizer=AutoTokenizer.from_pretrained(checkpoint )
Model=AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

def tokenize_function(example):
  return(Tokenizer(example["sentence1"], example["sentence2"],  truncation =True))



tokenized_dataset=data.map(tokenize_function, batched=True)
datacollator=DataCollatorWithPadding(tokenizer=Tokenizer)

# print(data['train'][5])
# print(tokenized_dataset['train'][5])



In [35]:
#training
# from transformers import TrainingArguments
# import numpy as np
# !pip install evaluate
# import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch"
    )

from transformers import Trainer
trainer = Trainer(Model, training_args,train_dataset=tokenized_dataset["train"],
                  eval_dataset=tokenized_dataset["validation"],
                  data_collator=datacollator, tokenizer=Tokenizer,
                  compute_metrics=compute_metrics
                  )
trainer.train()



Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.149444,0.821078,0.876897
2,No log,0.875263,0.840686,0.888889
3,0.072200,0.972407,0.848039,0.892361


TrainOutput(global_step=690, training_loss=0.06005738299825917, metrics={'train_runtime': 182.3809, 'train_samples_per_second': 60.335, 'train_steps_per_second': 3.783, 'total_flos': 430433242128000.0, 'train_loss': 0.06005738299825917, 'epoch': 3.0})

In [ ]:
# predictions = trainer.predict(tokenized_dataset["validation"])
